In [2]:
# import packages

import numpy as np 
import pandas as pd 
from scipy import signal
import matplotlib.pyplot as plt 
import math
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

---
# Data Loading and Preprocessing

In [17]:
# load datasets
def visualisation():
    for i in range(19):
        df = pd.read_csv('daliac/dataset_' + str(i+1) + '.txt', sep=',', header=None)
    for i in range(1, 14):
        df_exercise = df[df[24] == i].values
        plt.plot(df_exercise[500:1500, :])
        plt.show()

In [18]:
# remove noise
def removeNoise():
    for i in range(19):
        df = pd.read_csv('daliac/dataset_' + str(i+1) + '.txt', sep=',', header=None)
    for i in range(1,14):
        df_exercise = df[df[24] == i].values
        b, a = signal.butter(4, 0.04, 'lowpass', analog=False)
        for k in range(24):
            df_exercise[:,k] = signal.lfilter(b, a, df_exercise[:, k])
        plt.plot(df_exercise[500:1500, :])
        plt.show()

---
# Feature Engineering

In [26]:
def featureEngineering():    
    for i in range(19):
        df = pd.read_csv('daliac/dataset_' + str(i + 1) + '.txt', sep=',', header=None)
        print('deal with dataset ' + str(i + 1))
        for c in range(1, 14):
            activity_data = df[df[24] == c].values
            b, a = signal.butter(4, 0.04, 'low', analog=False)
            for j in range(24):
                activity_data[:, j] = signal.lfilter(b, a, activity_data[:, j])

                training = np.empty(shape=(0, 10))
                testing = np.empty(shape=(0, 10))
            

                datat_len = len(activity_data)
                training_len = math.floor(datat_len * 0.8)
                training_data = activity_data[:training_len, :]
                testing_data = activity_data[training_len:, :]
                training_sample_number = training_len // 1000 + 1
                testing_sample_number = (datat_len - training_len) // 1000 + 1

                for s in range(1):
                    if s < training_sample_number - 1 and s < testing_sample_number - 1:
                        sample_data = training_data[1000*s:1000*(s + 1), :]
                        sample_data = testing_data[1000*s:1000*(s + 1), :]
                    else:
                        sample_data = training_data[1000*s:, :]
                        sample_data = testing_data[1000*s:, :]

                    feature_sample = []
                    for i in range(3):
                        feature_sample.append(np.min(sample_data[:, i]))
                        feature_sample.append(np.max(sample_data[:, i]))
                        feature_sample.append(np.mean(sample_data[:, i]))
                    feature_sample.append(sample_data[0, -1])
                    feature_sample = np.array([feature_sample])
                    training = np.concatenate((training, feature_sample), axis=0)
                    testing = np.concatenate((testing, feature_sample), axis=0)

                    df_training = pd.DataFrame(training)
                    df_testing = pd.DataFrame(testing)
                    df_training.to_csv('training_data.csv', index=None, header=None)
                    df_testing.to_csv('testing_data.csv', index=None, header=None)

In [27]:
def model_training_and_evaluation_example():
    df_training = pd.read_csv('training_data.csv', header=None)
    df_testing = pd.read_csv('testing_data.csv', header=None)

    y_train = df_training[9].values
    y_train = y_train - 1
    df_training = df_training.drop([9], axis=1)
    X_train = df_training.values
    y_test = df_testing[9].values
    y_test = y_test - 1
    df_testing = df_testing.drop([9], axis=1)
    X_test = df_testing.values
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred)) 
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1,1e-2, 1e-3, 1e-4],
                     'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 100]},
                    {'kernel': ['linear'], 'C': [1e-3, 1e-2, 1e-1, 1, 10, 100]}]
    acc_scorer = make_scorer(accuracy_score)
    grid_obj  = GridSearchCV(SVC(), tuned_parameters, cv=10, scoring=acc_scorer)
    grid_obj  = grid_obj .fit(X_train, y_train)
    clf = grid_obj.best_estimator_
    print('best clf:', clf)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))



In [28]:
if __name__ == '__main__':
#     visualisation()
#     removeNoise()
    featureEngineering()
    model_training_and_evaluation_example()

deal with dataset 1
deal with dataset 2
deal with dataset 3
deal with dataset 4
deal with dataset 5
deal with dataset 6
deal with dataset 7
deal with dataset 8
deal with dataset 9
deal with dataset 10
deal with dataset 11
deal with dataset 12
deal with dataset 13
deal with dataset 14
deal with dataset 15
deal with dataset 16
deal with dataset 17
deal with dataset 18
deal with dataset 19


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 3